## Change detection using M3C2 algorithm

In [ ]:
import numpy as np
import time
import py4dgeo
import laspy

we need to load two datasets that cover the same scene at two different points in time.

In [ ]:
epoch1, epoch2 = py4dgeo.read_from_las(
    py4dgeo.find_file('../datasets/pointclouds/run1/sidewalk/processed_2386_9702.laz'),
    py4dgeo.find_file('../datasets/pointclouds/run2/sidewalk/processed_2386_9702.laz')
)

The distance analysis is executed on a number of points of interest called *core points*. This could be e.g. the entire reference point cloud, a downsampled version of it, an equistant grid etc. Here, we choose the entire reference point cloud:

In [ ]:
corepoints = epoch1.cloud

Next, we instantiate the algorithm class and run the distance calculation:

In [ ]:
"""
Some CloudCompare variables from m3c2_params.txt
See the mapping here: https://github.com/ssciwr/py4dgeo/issues/84
We have to divide these values by 2: https://github.com/ssciwr/py4dgeo/issues/128
"""
normal_scale = 0.70 / 2
search_scale = 0.70 / 2
max_distance = 3.55

start_time = time.time()

m3c2 = py4dgeo.M3C2(
    epochs=(epoch1, epoch2),
    corepoints=corepoints,
    cyl_radii=(search_scale,),
    normal_radii=(normal_scale,),
    max_distance=max_distance
)

distances, uncertainties = m3c2.run()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
def write_las(points, las_path, distances):
    """
    Saving the ndarray points data into a .las file.
    :param content: ndarray
    :param las_path: string, path to save the las file
    """
    print('Saving LAS lidar data')
    
    outfile = laspy.create(file_version="1.2", point_format=3)
    outfile.x = points[:, 0]
    outfile.y = points[:, 1]
    outfile.z = points[:, 2]
    if distances is not None:
        outfile.add_extra_dim(laspy.ExtraBytesParams(name="m3c2", type="float",
                              description="M3C2 distance"))
        outfile.m3c2 = distances
    outfile.write(las_path)

pointcloud = laspy.read('../datasets/pointclouds/run1/sidewalk/processed_2386_9702.laz')
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T

write_las(points, '../datasets/pointclouds/m3c2_test.laz', distances)